In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import time
import traceback
from datetime import datetime

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

In [2]:
input_directory = '/Users/sauravyadav/Documents/Repos/Datasets/Drinking_Surveys/output/prelim_data_0.csv'
submission_info_directory = "/Users/sauravyadav/Documents/Repos/Datasets/Drinking_Surveys/EMA Weekends_FINAL.csv"

In [3]:
df = pd.read_csv(input_directory)

/var/folders/35/8nzwnddx74qg4p54k0q2v_0r0000gn/T/ipykernel_21242/3544748107.py:1: DtypeWarning: Columns (28,29,30,31,32,33,34,35,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_directory)


### TO DO
- ~~Normalize Date Format~~
- ~~Float to Int~~
- ~~Create and Insert Columns : 
    - ~~'Brst': 'Burst'~~,
    - ~~'Wknd': 'Weekend'~~,
    - ~~'Day': 'Day Within Weekend'~~,
    - ~~'DrnkNum_combine',
    - ~~'DrnkNum_cumulative',
- MR_DrnkDur column
- MR_wake column
- ~~Consider additional columns to add:
    - ~~Date of Drinking Session
    - ~~Group
    

### Uniform Date Format

In [4]:
date_columns = [col for col in df.columns if 'date' in col.lower()]
formats = ['%d/%m/%Y', '%Y-%d-%m %H:%M:%S']
# Function to try parsing the date with multiple formats
def parse_date(date_str):
    for fmt in formats:
        try:
            return pd.to_datetime(date_str, format=fmt)
        except ValueError:
            # print(f'String: {date_str} Format failure: {fmt}')
            pass
    print(date_str)
    return pd.NaT  # Return NaT (Not-a-Time) for unparseable dates

# df[(df['TrigDate'].str.contains('-'))]['TrigDate'].sort_values(ascending=False)
# df['TrigDate'].sample(5)

# Apply the parse_date function to the 'DateColumn'
for date_col in date_columns:
    df[date_col] = df[date_col].apply(parse_date)

### Remove Decimal Points

In [5]:
int_columns = ['UI_DrnkFin', 'UI_DrnkNum', 'DrnkNum', 'Plsur', 'Rliev', 'IntxNow',
       'MorAlc', 'enrgz', 'excit', 'sedat', 'slotht', 'slug', 'up', 'crntloc',
       'wthothr', 'NumOth', 'OthTyp', 'OthWho', 'EnjInt', 'vrtint', 'vrtoth', 'vrtwho', 'vrtenj', 'ActCld',
       'ActAcc', 'OthCld', 'OthAcc', 'Vrtl', 'AlnLke', 'AlnGd', 'WshOth',
       'lnly', 'rjct', 'incl', 'acpt', 'vrtcld', 'VrtAcc', 'VrtOthCld',
       'VrtOthAcc', 'NotInt', 'WshInt', 'UI_NtInt_lnly', 'UI_NtInt_rjct',
       'UI_NtInt_incl', 'UI_NtInt_acpt', 'said', 'spent', 'impt', 'spur']

for col in int_columns:
    df[col] = df[col].astype('Int64', errors='ignore')

### Convert Date to UTC

In [6]:
df['SubDate'] = df['SubDate'].astype('datetime64[ns]')

### New Columns: Burst,Weekend, Day Within Weekend

In [7]:
date_info_df = pd.read_csv(submission_info_directory)

In [8]:
date_info_df = date_info_df[~date_info_df['Sub ID'].isna()].reset_index(drop=True)
date_info_df['SubID'] = date_info_df['Sub ID'].astype('Int64', errors='ignore')
date_info_df = date_info_df.drop(columns=['Date of Drinking Session ', 'Group', 'Sub ID']).reset_index(drop=True)
date_info_df.sample()

,EMA Weekend 1,EMA Weekend 2,EMA Weekend 3,EMA Weekend 4,EMA Weekend 5,EMA Weekend 6,SubID
318,Thurs 4/28 - Sun 5/1/2022,Thurs 5/5 - Sun 5/8/2022,Thurs 10/20 - Sun 10/23/22,Thurs 10/27 - Sun 10/30/22,Thu 4/20 - Sun 4/23/23,Thu 4/27 - Sun 4/30/23,2678


In [9]:
date_info_df = pd.melt(date_info_df, id_vars=['SubID'], var_name='Weekend', value_name='Submission_Range')
date_info_df['Weekend'] = date_info_df['Weekend'].str.replace('EMA Weekend', '').astype(int)

def expand_year(d):
    date_parts = d.split('/')
    if len(date_parts)==3:
        year = date_parts[2]
        if len(year)==2:
            year = f'20{year}'
            d = f'{d[:-2]}{year}'
    return d
    
def expand_date_range(row):
    
    try:
        if isinstance(row['Submission_Range'], datetime):
            return []
        date_parts = row['Submission_Range'].split('-')
        if len(date_parts)<=1:
            return []
        start_date_parts = date_parts[0].strip().split(' ')
        end_date_parts = date_parts[1].strip().split(' ')
        
        start_date = start_date_parts[1]
        end_date = end_date_parts[1] if len(end_date_parts)==2 else end_date_parts[0]
        
        start_date = expand_year(start_date)
        end_date = expand_year(end_date)
            
        if len(start_date.split('/'))==2 and len(end_date.split('/'))==2:
            print(f'SubID: {row["SubID"]} {row["Submission_Range"]} Range End Date Year: {end_date} Start Year: {start_date}')
            return []
        elif len(start_date.split('/'))==2:
            end_date_month, end_date_date ,end_date_year = end_date.split('/')
            start_date_year = end_date_year 
            if end_date_month=='1' and end_date_date in['1','2','3']:
                start_date_year = str(int(end_date_year) - 1)
            start_date = start_date + f'/{start_date_year}'
        
        elif len(end_date.split('/'))==2:
            start_date_month, start_date_date, start_date_year = start_date.split('/')
            end_date_year = start_date_year
            if start_date_month=='12' and start_date_date in ['29','30','31']:
                end_date_year = str(int(start_date_year)+1)
            
            end_date = end_date + f'/{end_date_year}'
        
        try:
            start_date = pd.to_datetime(start_date, format='%m/%d/%Y')
            end_date = pd.to_datetime(end_date, format='%m/%d/%Y')
        except ValueError:
            print(f'{start_date} and {end_date}')
            
        # date_range = pd.date_range(start_date, end_date, freq='D')
        date_range = pd.date_range(start_date, periods=4, freq='D')
        res = [date.strftime('%a %m/%d/%y') for date in date_range]
        
        if len(res)!=4:
            print(row)
            print(res)
            print(f"!! RANGE IS {row['Submission_Range']}, DATES ARE {start_date} and {end_date}")
        
        return res
    except Exception as e:
        traceback.print_exc()
        display(row)
        print(e)
        return []

date_info_df['Submission_Date'] = date_info_df.apply(expand_date_range, axis=1)
date_info_df = date_info_df.explode('Submission_Date').reset_index(drop=True)
date_info_df['Burst'] = np.ceil(date_info_df['Weekend'] / 2).astype(int)
date_info_df['Day'] = pd.to_datetime(date_info_df['Submission_Date'], format='%a %m/%d/%y').dt.strftime('%a')
date_info_df = date_info_df[~date_info_df['Submission_Date'].isna()]
date_info_df['Submission_Date'] = date_info_df['Submission_Date'].astype('datetime64[ns]')
date_info_df = date_info_df.drop(columns=['Submission_Range'])
# date_info_df = date_info_df.drop(columns=['Submission_Range', 'Submission_Date']).reset_index(drop=True)

Traceback (most recent call last):
  File "/var/folders/35/8nzwnddx74qg4p54k0q2v_0r0000gn/T/ipykernel_21242/660349064.py", line 24, in expand_date_range
    start_date = start_date_parts[1]
                 ~~~~~~~~~~~~~~~~^^^
IndexError: list index out of range


SubID                 2207
Weekend                  5
Submission_Range    Oct-19
Name: 1683, dtype: object

list index out of range


Traceback (most recent call last):
  File "/var/folders/35/8nzwnddx74qg4p54k0q2v_0r0000gn/T/ipykernel_21242/660349064.py", line 24, in expand_date_range
    start_date = start_date_parts[1]
                 ~~~~~~~~~~~~~~~~^^^
IndexError: list index out of range


SubID                 2228
Weekend                  5
Submission_Range    Oct-19
Name: 1688, dtype: object

...

list index out of range


Traceback (most recent call last):
  File "/var/folders/35/8nzwnddx74qg4p54k0q2v_0r0000gn/T/ipykernel_21242/660349064.py", line 24, in expand_date_range
    start_date = start_date_parts[1]
                 ~~~~~~~~~~~~~~~~^^^
IndexError: list index out of range


SubID                 2222
Weekend                  6
Submission_Range    Oct-19
Name: 2084, dtype: object

list index out of range


/var/folders/35/8nzwnddx74qg4p54k0q2v_0r0000gn/T/ipykernel_21242/660349064.py:75: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  date_info_df['Submission_Date'] = date_info_df['Submission_Date'].astype('datetime64[ns]')


In [10]:
res_df = df.merge(date_info_df, left_on=['SubID','SubDate'], right_on=['SubID', 'Submission_Date'], how = 'right')

In [11]:
res_df.head(1)

,SubID,RespID,RespType,UserID,SurvName,SurvType,InitDate,InitTime,SubDate,SubTime,TrigDate,TrigTime,UI_DrnkFin,UI_DrnkNum,DrnkNum,Plsur,Rliev,IntxNow,MorAlc,enrgz,excit,sedat,slotht,slug,up,crntloc,othrloc,wthothr,NumOth,OthTyp,OthWho,EnjInt,ActCld,ActAcc,OthCld,OthAcc,Vrtl,AlnLke,AlnGd,WshOth,lnly,rjct,incl,acpt,vrtint,vrtoth,vrtwho,vrtenj,vrtcld,VrtAcc,VrtOthCld,VrtOthAcc,NotInt,WshInt,UI_NtInt_lnly,UI_NtInt_rjct,UI_NtInt_incl,UI_NtInt_acpt,said,spent,impt,spur,SurvNamePlus,Weekend,Submission_Date,Burst,Day
0,2013,5aab22d4c472b43d9846c725,Submission,5a9844f3267c2b1d45271ad3,After Your 1st Drink,Assessment,2018-03-15,21:49:09,2018-03-15,21:50:10,2018-03-15,05:00:00,4,1,<NA>,32.0,33.0,14,13,21,36,34,28,27,74,4,CONDITION_SKIPPED,1,3,3,CONDITION_SKIPPED,65,67,61,58,58,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,31,18,16,18,Start,1,2018-03-15,1,Thu


In [12]:
# print(res_df[res_df['Day'].isna()].shape)
# print(res_df[res_df['Day'].isna()]['SurvName'].value_counts())
# print(res_df[res_df['Day'].isna()]['RespType'].value_counts())
# print(df.shape)
display(res_df[res_df['Day'].isna()].head())

,SubID,RespID,RespType,UserID,SurvName,SurvType,InitDate,InitTime,SubDate,SubTime,TrigDate,TrigTime,UI_DrnkFin,UI_DrnkNum,DrnkNum,Plsur,Rliev,IntxNow,MorAlc,enrgz,excit,sedat,slotht,slug,up,crntloc,othrloc,wthothr,NumOth,OthTyp,OthWho,EnjInt,ActCld,ActAcc,OthCld,OthAcc,Vrtl,AlnLke,AlnGd,WshOth,lnly,rjct,incl,acpt,vrtint,vrtoth,vrtwho,vrtenj,vrtcld,VrtAcc,VrtOthCld,VrtOthAcc,NotInt,WshInt,UI_NtInt_lnly,UI_NtInt_rjct,UI_NtInt_incl,UI_NtInt_acpt,said,spent,impt,spur,SurvNamePlus,Weekend,Submission_Date,Burst,Day


In [13]:
res_df['Day'] = res_df.apply(lambda row: (row['SubDate'].strftime('%a')) if pd.isna(row['Day']) else row['Day'], axis = 1)

In [14]:
# print(res_df[res_df['Day'].isna()].head())
# # display(res_df[res_df['SubID'] == 2123].head(100))
# print(f'Subs: {date_info_df['SubID'].nunique()}\nShape: {date_info_df.shape}')
# print(f'Subs: {res_df['SubID'].nunique()}\nShape: {res_df.shape}')

In [15]:
res_df = res_df.drop(columns=['Submission_Date'])

### New Columns: 'DrnkNum_combine', 'DrnkNum_cumulative'

In [16]:
# res_df['DrnkNum_combine'] = res_df.apply(lambda x: x['DrnkNum'] - 1 if (pd.notnull(x['DrnkNum']) and x['DrnkNum'] > 1 and x['RespType'] != 'Missed' and x['SurvName'] == 'Drinking Follow-Ups')  else (x['UI_DrnkNum'] if (x['SurvName'] == 'After Your 1st Drink' and x['RespType'] != 'Missed')  else (0 if (pd.notnull(x['DrnkNum']) and x['DrnkNum'] == 1 and x['RespType'] != 'Missed') else None)), axis = 1)
# res_df['DrnkNum_combine'] = res_df['DrnkNum_combine'].astype('Int64')

In [17]:
res_df = res_df.sort_values(by=['SubID', 'SubDate', 'SubTime', 'SurvName']).reset_index(drop=True)

In [18]:
# grouped = res_df.groupby(['SubID', 'Weekend', 'Burst'])
# res_df['DrnkNum_cumulative'] = None

# for group_name, group_df in grouped:
#     cumulativeDrinkNum = 0
#     for index, row in group_df.iterrows():
        
#         if row['SurvName'] == 'After Your 1st Drink':
#             cumulativeDrinkNum = 0

#         if row['RespType'] == 'Missed':
#             continue
            
#         cumulativeDrinkNum += 0 if pd.isna(row['DrnkNum_combine']) else row['DrnkNum_combine']
#         res_df.at[index, 'DrnkNum_cumulative'] = cumulativeDrinkNum

### Finishing up

In [19]:
res_df = res_df[['SubID','Burst', 'Weekend', 'Day', 'RespID', 'RespType', 'UserID', 'SurvName', 'SurvNamePlus', 'SurvType',
       'InitDate', 'InitTime', 'SubDate', 'SubTime', 'TrigDate', 'TrigTime',
       'UI_DrnkFin', 'UI_DrnkNum', 'DrnkNum', 'Plsur', 'Rliev', 'IntxNow',
       'MorAlc', 'enrgz', 'excit', 'sedat', 'slotht', 'slug', 'up', 'crntloc',
       'othrloc', 'wthothr', 'NumOth', 'OthTyp', 'OthWho', 'EnjInt', 'ActCld',
       'ActAcc', 'OthCld', 'OthAcc', 'Vrtl', 'AlnLke', 'AlnGd', 'WshOth',
       'lnly', 'rjct', 'incl', 'acpt','vrtint', 'vrtoth',
        'vrtwho', 'vrtenj', 'vrtcld', 'VrtAcc', 'VrtOthCld',
       'VrtOthAcc', 'NotInt', 'WshInt', 'UI_NtInt_lnly', 'UI_NtInt_rjct',
       'UI_NtInt_incl', 'UI_NtInt_acpt', 'said', 'spent', 'impt', 'spur']]

In [20]:
res_df.to_csv('/Users/sauravyadav/Documents/Repos/Datasets/Drinking_Surveys/output/data_draft.csv', index=False)

In [21]:
# res_df[res_df['Burst'].isna()].head()

,SubID,Burst,Weekend,Day,RespID,RespType,UserID,SurvName,SurvNamePlus,SurvType,InitDate,InitTime,SubDate,SubTime,TrigDate,TrigTime,UI_DrnkFin,UI_DrnkNum,DrnkNum,Plsur,Rliev,IntxNow,MorAlc,enrgz,excit,sedat,slotht,slug,up,crntloc,othrloc,wthothr,NumOth,OthTyp,OthWho,EnjInt,ActCld,ActAcc,OthCld,OthAcc,Vrtl,AlnLke,AlnGd,WshOth,lnly,rjct,incl,acpt,vrtint,vrtoth,vrtwho,vrtenj,vrtcld,VrtAcc,VrtOthCld,VrtOthAcc,NotInt,WshInt,UI_NtInt_lnly,UI_NtInt_rjct,UI_NtInt_incl,UI_NtInt_acpt,said,spent,impt,spur


### See next jupyter part....